In [ ]:
"""
Thread

    t = threading.Thread(target=func, args=[...], kwargs={...}) 类可以创建一个进程对象
        - target 进程入口函数
        - args 进程入口函数的参数列表
        - kwargs 进程入口函数参数字典
        
    t.start() 启动进程
    t.join() 等待进程执行结束
"""

from threading import Thread

total = 0

def func(times):
    global total

    for x in range(times):
        total += 1


t = Thread(target=func, kwargs=dict(times=10))
t.start()
t.join()

print('* after threading finish, result is {}'.format(total))

In [ ]:
"""
Thread

    从 Thread 类继承后，也可以完成线程，实现 run 方法即可
"""

from threading import Thread

class Runner(Thread):
    def __init__(self, times):
        super().__init__()
        self._n, self._times = 0, times

    @property
    def n(self):
        return self._n

    def run(self):
        for x in range(self._times):
            self._n += 1


r1 = Runner(10)
r2 = Runner(20)
r1.start()
r2.start()
r1.join()
r2.join()

print('* after threading finish, result r1.n is {}'.format(r1.n))
print('* after threading finish, result r2.n is {}'.format(r2.n))

In [ ]:
"""
Sleep

    time.sleep(sec) 可以设定线程休眠的时间, 时间是一个表示秒的浮点数
"""

from threading import Thread
from time import sleep
from timeit import default_timer as timer

def runner(sleep_time_):
    sleep(sleep_time_)


sleep_time = 0.5

t = Thread(target=runner, args=(sleep_time,))

start = timer()

t.start()
t.join()

print('* after sleep {} sec, thread run {:.2f} sec'.format(sleep_time, timer() - start))

In [ ]:
"""
active_count

    获取当前有多少个线程处于活动状态
"""

from threading import Thread, Condition, active_count


print('* current active threading count is {}'.format(active_count()))

def func(cond_):
    with cond_:
        cond_.wait()


cond = Condition()

t = Thread(target=func, args=(cond,))
t.start()

print('* current active threading count is {}'.format(active_count()))

with cond:
    cond.notify()

t.join()

In [ ]:
"""
Lock

    threading.Lock() 用于创建一个互斥锁
    
    lock = threading.Lock() # 获得锁对象
    lock.acquire() # 占用锁
    try:
        ... # 临界区
    finally:
        lock.release() # 释放所
        
    或
    
    with lock:
        ... # 临界区
"""

from threading import Thread, Lock

name = None

def func(lock_):
    print('* lock is locked? {}'.format(lock_.locked()))

    with lock_:
        global name

        print('* in thread name is {}'.format(name))
        name = 'Hello, {}'.format(name)

    print('* lock is locked? {}'.format(lock_.locked()))


lock = Lock()

with lock:
    t = Thread(target=func, args=(lock,))
    t.start()

    name = 'Alvin'

with lock:
    print('* after thread dead, name is {}'.format(name))

In [ ]:
"""
RLock

    RLock 和 Lock 功能类似，主要区别在于，在同一个线程内，Lock 对象只能 acquire 一次，而 RLock 对象可以 acquire 多次。
    简单理解就是：RLock 在同一个线程内不进行锁定
    
    注意：RLock 每次 acquire 都需要对应的 release 来释放。
"""

from threading import Thread, Lock, RLock

lock = Lock()

r = lock.acquire(blocking=False)
print('* with Lock, first acquire result is {}'.format(r))

r = lock.acquire(blocking=False)
print('* with Lock, second acquire in same thread result is {}'.format(r))

lock.release()

lock = RLock()

r = lock.acquire(blocking=False)
print('* with RLock, first acquire result is {}'.format(r))

r = lock.acquire(blocking=False)
print('* with RLock, second acquire in same thread result is {}'.format(r))

lock.release()
lock.release()

In [ ]:
"""
Condition

    threading.Condition() 在 Lock 的基础上增加了等待和通知功能。
    
    cond = threading.Condition()
    
    # in one thread:
    cond.acquire()
    try:
        cond.wait(1)
    finally:
        cond.release()
        
    # in other thread
    cond.acquire()
    try:
        cond.notify()
    finally:
        cond.release()
        
    wait 和 notify 方法都只能在当前 Condition 临界区内调用，即需要先调用 acquire 方法
    wait 方法会暂时退出临界区，wait 结束后（成功或超时）会回到临界区
    
    上述流程可简化为
    with cond:
        cond.wait(1)
        
    with cond:
        cond.notify()
"""

from time import sleep
from threading import Thread, Condition

def func(id_, cond_):
    with cond_:
        if cond_.wait(1):
            print('* thread {} wait success'.format(id_))
        else:
            print('* thread {} wait timeout'.format(id_))


cond = Condition()

for t in [Thread(target=func, args=('A', cond,)), Thread(target=func, args=('B', cond,))]:
    t.start()

sleep(0.5)

with cond:
    cond.notify()

for t in [Thread(target=func, args=('A', cond,)), Thread(target=func, args=('B', cond,))]:
    t.start()

sleep(0.5)

with cond:
    cond.notify_all()

In [ ]:
"""
Semaphore

    threading.Semaphore() 得到一个信号量
"""

from threading import Thread, Semaphore, RLock
from time import sleep

class Printer:
    def __init__(self):
        self._lock = RLock()

    def print_resource_get(self, id_, res):
        s = '* thread {} get resource {}'.format(id_, res)
        with self._lock:
            print(s)

    def print_resource_back(self, id_, res):
        s = '* thread {} put back resource {}'.format(id_, res)
        with self._lock:
            print(s)


printer = Printer()


res = list(range(5))
print('* total resources are: {}'.format(res))

def func(id_, semp_):
    with semp_:
        r = res.pop()  # 获取资源
        printer.print_resource_get(id_, r)

        sleep(1)  # 消费资源

        res.append(r)  # 归还资源
        printer.print_resource_back(id_, r)


semp = Semaphore(len(res))

threads = [Thread(target=func, args=(chr(ord('A') + n), semp)) for n in range(20)]

for t in threads:
    t.start()

for t in threads:
    t.join()

In [ ]:
"""
starmap

    with ThreadPool(processes=n_threads) as pool:
        pool.starmap(func, iterable) 指定方法分配到不同进程执行
            - func 进程入口函数；
            - iterable 为一个迭代器，表示 func 函数如何获取参数
        
    例如有如下函数：
        def func(a, b, c):
            ...
            
    则：
        pool.starmap(func, [(a1, b1, c1), (a2, b2, c2), (a3, b3, c3)]) 表示：
        通过三个进程执行 func 函数，每个进程传递一组参数
"""

from multiprocessing.pool import ThreadPool
from multiprocessing import cpu_count
import timeit as ti
import time as t

n_threads = cpu_count() * 2

def is_prime(n):
    if n <= 1:
        return False

    for i in range(2, n):
        if n % i == 0:
            return False
        t.sleep(0.1)
    return True


with ThreadPool(processes=n_threads) as pool:
    start = ti.default_timer()
    results = pool.starmap(is_prime, zip(range(10)))

cost = ti.default_timer() - start
r = list(enumerate(results))
print('* prime numbers between 0~100 are {}, cast {:.5f} sec'.format(r, cost))

In [ ]:
"""
apply, apply_async

    with ThreadPool(processes=n_threads) as pool:
        pool.apply(func, args) 在进程池内放置一个函数调用，并等待其执行完毕，返回执行结果
            - func 进程入口函数；
            - args func 函数的参数
        
        pool.apply_async 函数和 apply 函数类似，但不等待执行完毕，返回调用句柄
        
    例如有如下函数：
        def func(a, b, c):
            ...
            
    则：
        pool.apply(func, args=(a1, b1, c1)) 表示：
        将 func 函数放到进程池中执行，传递参数 a1, b1, c1
        
    如果使用
        h = pool.apply_async(func, args=(a1, b1, c1))
        则必须获取一次结果
        result = h.get()
"""

from multiprocessing.pool import ThreadPool
from multiprocessing import cpu_count
import timeit as ti
import time as t


n_threads = cpu_count()

def is_prime(n):
    if n <= 1:
        return False

    for i in range(2, n):
        if n % i == 0:
            return False
        t.sleep(0.1)
    return True


results = [False] * 10
with ThreadPool(processes=n_threads) as pool:
    start = ti.default_timer()

    for i in range(len(results)):
        results[i] = pool.apply(is_prime, args=(i,))

cost = ti.default_timer() - start
r = list(enumerate(results))
print('* prime numbers between 0~100 are {}, cast {:.5f} sec'.format(r, cost))

In [ ]:
"""
map, map_async

    with ThreadPool(processes=n_threads) as pool:
        pool.map(func, iterable) 在进程池内放置一个函数调用，并等待其执行完毕，返回执行结果
            - func 进程入口函数；
            - iterable 一个序列，表示 func 函数的参数
        
        pool.map_async 函数和 map 函数类似，但不等待执行完毕，返回调用句柄
        
    例如有如下函数：
        def func(a):
            ...
            
    则：
        pool.map(func, [a1, a2, a3, a4]) 表示：
        将 func 函数放到进程池中执行，依次传递参数 a1, a2, a3
        
    如果使用
        h = pool.map_async(func, [a1, a2, a3, a4])
        则必须获取一次结果
        result = h.get()
"""

from multiprocessing.pool import ThreadPool
from multiprocessing import cpu_count
import timeit as ti
import time as t


n_threads = cpu_count()

def is_prime(n):
    if n <= 1:
        return False

    for i in range(2, n):
        if n % i == 0:
            return False
        t.sleep(0.1)
    return True


with ThreadPool(processes=n_threads) as pool:
    start = ti.default_timer()
    results = pool.map(is_prime, range(10))

cost = ti.default_timer() - start
r = list(enumerate(results))
print('* prime numbers between 0~100 are {}, cast {:.5f} sec'.format(r, cost))

In [ ]:
"""
ThreadPoolExecutor

    executor = concurrent.futures.ThreadPoolExecutor(max_workers=n, thread_name_prefix='', initializer=None, initargs=())
        - max_workers workers 的数量
        - thread_name_prefix 线程名称
        - initializer 任务初始化入口
        - initargs 任务初始化参数
        
    future = executor.submit(func, *args, **kwargs) # 提交一个任务到进程池
        - func 进程入口函数；
        - args/kwargs 进程入口函数参数；
        
    executor.shutdown(wait=True) # 关闭进程池
        - wait 是否等待所有任务结束
        
    concurrent.futures.wait(futures) # 等待所有的任务结束
    
    # 简化写法
    with ProcessPoolExecutor(n_process) as executor:
        ...
        
    executor.map(fn, *iterables, timeout=None, chunksize=1)
        - fn 进程方法入口
        - iterables 每个进程入口的参数序列
        - timeout 执行超时时间
"""

from concurrent.futures import ThreadPoolExecutor, wait
from multiprocessing import cpu_count
from time import sleep
from timeit import default_timer as timer


n_threads = cpu_count()

def func(id_):
    sleep(1)
    return 'task {} was completed'.format(id_)


executor = ThreadPoolExecutor(n_threads)

start = timer()
futures = [executor.submit(func, n) for n in range(n_threads * 2)]
return_futures = wait(futures, timeout=1)
end = timer() - start

print('* there are {}/{} task was done'.format(len(return_futures.done), len(futures)))
print('* there are {}/{} task was not done'.format(len(return_futures.not_done), len(futures)))

print('* done task results: ')
for f in return_futures.done:
    print('\t{}'.format(f.result()))

print('* all tasks cost {:.2f} sec'.format(end))

print()

start = timer()
futures = executor.map(func, range(n_threads * 2))

results = [r for r in futures]

print('* results are: ')
for r in results:
    print('\t{}'.format(r))

end = timer() - start
print('* all tasks cost {:.2f} sec'.format(end))

In [ ]:
"""
Thread local
"""

from threading import Thread, local, current_thread

loc = local()
loc.n = 100

def func():
    loc.n = 200
    print('* in thread {} \'loc.n\' is {}'.format(current_thread().name, loc.n))


t = Thread(target=func)
t.start()
t.join()

print('* in thread {} \'loc.n\' is {}'.format(current_thread().name, loc.n))

In [ ]:
"""
Werkzeug thread local
"""

from werkzeug.local import Local
from threading import Thread, current_thread

local = Local()
local.n = 100

def func():
    local.n = 200
    print('* in thread {} \'loc.n\' is {}'.format(current_thread().name, local.n))


t = Thread(target=func)
t.start()
t.join()

print('* in thread {} \'loc.n\' is {}'.format(current_thread().name, local.n))

In [ ]:
"""
Werkzeug thread local proxy
"""

from werkzeug.local import Local, LocalProxy
from threading import Thread, current_thread

class Holder:
    def __init__(self, n=0):
        self.n = n


loc = Local()
holder = LocalProxy(loc, 'holder')

loc.holder = Holder(0)

def func():
    loc.holder = Holder(100)
    print('* in thread {} \'loc.n\' is {}'.format(current_thread().name, holder.n))


t = Thread(target=func)
t.start()
t.join()

print('* in thread {} \'holder.n\' is {}'.format(current_thread().name, holder.n))

In [ ]:
"""
Werkzeug thread local proxy
"""

from werkzeug.local import Local, LocalProxy
from threading import Thread, current_thread

loc = Local()

class Holder:
    def __init__(self):
        self.n = 0

def get_local_value():
    if not hasattr(loc, 'holder'):
        loc.holder = Holder()
    return loc.holder


holder = LocalProxy(get_local_value)
holder.n = 0

def func():
    holder.n = 100
    print('* in thread {} \'loc.n\' is {}'.format(current_thread().name, holder.n))


t = Thread(target=func)
t.start()
t.join()

print('* in thread {} \'holder.n\' is {}'.format(current_thread().name, holder.n))

In [ ]:
"""
Werkzeug thread local proxy
"""

from werkzeug.local import Local, LocalProxy
from threading import Thread, current_thread

class Holder:
    def __init__(self, n=0):
        self.n = n


loc = Local()
holder = loc('holder')  # shortcuts for LocalProxy(loc, 'holder')

loc.holder = Holder(0)

def func():
    loc.holder = Holder(100)
    print('* in thread {} \'loc.n\' is {}'.format(current_thread().name, holder.n))


t = Thread(target=func)
t.start()
t.join()

print('* in thread {} \'holder.n\' is {}'.format(current_thread().name, holder.n))